# WhisperTS joining JSON to SRT

In [16]:
import re,codecs,unidecode
import pandas as pd
import networkx as nx
import matplotlib.pylab as p

import json,glob,os
nMachine=os.uname()[1]
if "iMac" in nMachine:
    nMachine="iMac"
elif "MacBook" in nMachine:
    nMachine="MacBook"
print(nMachine)

MacBook


In [17]:
def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [18]:
repJson="/Users/gilles/sDrive/Cours/Bordeaux/L2-Corpus/Whisper/%s/Json/"%nMachine
segments=len(glob.glob(repJson+"*.json"))
segments

3

In [19]:
json1=repJson+"extrait-00.json"
with open(json1) as inFile:
    whisper1=json.load(inFile)

### Gestion des timestamps

In [20]:
def second2timestamp(seconds):
    sHours=int(seconds/3600)
    sMinutes=int((seconds-sHours*3600)/60)
    sSeconds=int((seconds-sHours*3600-sMinutes*60))
    sMilliseconds=int((seconds%1)*1000)
    return "%02d:%02d:%02d,%03d"%(sHours,sMinutes,sSeconds,sMilliseconds)

def timestamp2second(timestamp):
    m=re.match("(\d\d):(\d\d):(\d\d),(\d\d\d)",timestamp)
    if m:
        seconds=(int(m.group(1))*60+int(m.group(2)))*60+int(m.group(3))+float(m.group(4))/1000
        return seconds
    else:
        print("problème de timestamp")
        return

def shiftTime(timestamp,decal):
    return second2timestamp(timestamp2second(timestamp)+decal)

In [21]:
punct = list('''!()[]{};:-'"\,<>./?@#$%^&*_~''')
maxSize=8

def removePunct(texte):
    result=texte
    for p in punct:
        result=result.replace(p,"")
    return result

### Gestion des structures JSON de Whisper

In [22]:
def getWords(whisper):
    lWords=[]
    for segment in whisper["segments"]:
        for word in segment["words"]:
            lWords.append((second2timestamp(word["start"]),
                              second2timestamp(word["end"]),
                              word["text"],
                              word["confidence"]))        
    return [(t1,t2,w,c) for t1,t2,w,c in lWords if w!=""]

def getList(tWhisper):
    return [unidecode.unidecode(removePunct(w)).lower() for _,_,w,_ in tWhisper]

In [23]:
for s in whisper1['segments']:
    for w in s["words"]:
        if "start" not in w:
            print(w)

In [24]:
srt=getWords(whisper1)
srtList=getList(srt)

### Trouver la jointure

In [25]:
def findSample(sample,ref):
    lowSample=[t.lower() for t in sample]
    lowRef=[t.lower() for t in ref]
    i=0
    while i<len(ref)-len(sample):
        # print("ref",i)
        if lowSample==lowRef[i:i+len(sample)]:
            return i
        else:
            i+=1
    return None

### Collage des mots
1. on cherche la partie commune entre le nouveau segment et les sous-titres précédents
2. on peut régler la longueur de la partie commune pour optimiser le collage *maxSize*
3. on incrémente les temps de la nouvelle partie à partir du deuxième mot (pour éviter les erreurs de timestamp en début d'enregistrement)

In [26]:
def pasteWords(srt,whisper2):
    srtList=getList(srt)
    w2=getWords(whisper2)
    w2List=getList(w2)
    lenW2=len(w2)
    compSize=min(maxSize,lenW2)
    i=1
    print(compSize)
    while i<lenW2-compSize:
        sample=w2List[i:i+compSize]
        ref=srtList
        # print(sample)
        idx=findSample(sample,ref)
        if idx:
            print("found",i,idx)
            break
        else:
            i+=1
    if idx:
        print("found",i,idx,srtList[idx],srt[idx][0],w2List[i],w2[i][0])
        tDecal=timestamp2second(srt[idx+1][0])-timestamp2second(w2[i+1][0])
        print(second2timestamp(tDecal))
        newSrt=[]
        for n in range(i+1,lenW2):
            nWord=w2[n]
            newWord=(shiftTime(nWord[0],tDecal),shiftTime(nWord[1],tDecal),nWord[2],nWord[3])
            newSrt.append(newWord)
        print(srt[idx+1],newSrt[0])
        return srt[:idx]+newSrt
    else:
        print("problème")
        ding()
        ding()
        ding()
        return None

In [27]:
if segments:
    for i in range(1,segments):
        print(i)
        json2=repJson+"extrait-%02d.json"%i
        with open(json2) as inFile:
            whisper2=json.load(inFile)
        newSrt=pasteWords(srt,whisper2)
        if not newSrt:
            break
        else:
            srt=newSrt
# print(srt)

1
8
found 15 206
found 15 206 cest 00:01:03,979 cest 00:00:04,000
00:01:00,000
('00:01:04,099', '00:01:04,159', 'mon', 0.227) ('00:01:04,099', '00:01:04,179', 'mon', 0.831)
2
8
found 1 379
found 1 379 si 00:02:01,129 si 00:00:01,590
00:02:00,019
('00:02:01,750', '00:02:02,049', 'les', 0.895) ('00:02:01,750', '00:02:02,048', 'les', 0.87)


In [28]:
pause=0.200

srtTours=[]
srtMots=[]
bTour=True
prevEnd=0
pStart=second2timestamp(0)
pEnd=0

for i,(start,end,mot,confidence) in enumerate(srt):
    # print(start,end,mot,confidence)
    tStart=timestamp2second(start)
    tEnd=timestamp2second(end)
    # print(start,tStart,prevEnd)
    if tStart-prevEnd>pause and prevEnd!=0:
        pEnd=second2timestamp(prevEnd)
        pTour=" ".join(srtMots)
        srtTours.append((pStart,pEnd,pTour))
        pStart=start
        prevEnd=tEnd
        srtMots=[mot]
        bPhrase=True
    else:
        srtMots.append(mot)
        prevEnd=tEnd
pTour=" ".join(srtMots)
srtTours.append((pStart,pEnd,pTour))

In [29]:
ding()

In [30]:
srtLines=[]
for i,(start,end,mot) in enumerate(srtTours):
    print(start,end,mot)
    n=i+1
    srtLines.append(str(n))
    srtLines.append(start+" --> "+end)
    srtLines.append(mot)
    srtLines.append("")


00:00:00,000 00:00:06,820 Moi je conseillerai jamais à un jeune [*] de [*] vivre et de faire ce que j'ai fait parce que [*] c'est [*] vraiment dur.
00:00:12,640 00:00:21,018 Alors voilà, il y a une [*] légende [*] de Quantonat, fait à la fois de [*] génie du football et à la fois de [*] rébellion contre l'ordre établi,
00:00:21,500 00:00:23,980 et c'est pour ça que [*] vous êtes le bienvenu.
00:00:25,059 00:00:26,719 Et alors dans cette famille Quantonat,
00:00:26,940 00:00:28,879 [*] il y avait un enclairmite,
00:00:29,280 00:00:32,320 et c'est important pour la suite parce que ça [*] vous a sûrement beaucoup influencé,
00:00:33,100 00:00:34,759 qui vivait au fin fond de l'ardèche,
00:00:35,520 00:00:37,280 qui avait une barbe jusqu'au nombril,
00:00:37,520 00:00:41,259 qui avait des chèvres, qui assommait les truites qui pêchaient dans la rivière sur des rochers,
00:00:41,840 00:00:46,219 qui mangeaient des fruits qui cueillaient, des salades des prêts, qui buvaient l'eau des rivière

In [31]:
with codecs.open(json1.replace(".json","-tours.srt"),"w",encoding="utf8") as outFile:
    for srtLine in srtLines:
        outFile.write(srtLine+"\n")